In [1]:
import json
import pandas as pd
import numpy as np
import re
import sys
import sqlalchemy.types as tp
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date
from sqlalchemy.orm import sessionmaker

sys.path.append('..')

In [2]:
from helper import get_database_session, init_mysql_engine

In [3]:
with open('../config.json', 'r') as f:
  CONFIG = json.load(f)

engine = init_mysql_engine(CONFIG['MYSQL_USER'],
                           CONFIG['MYSQL_PWD'],
                           CONFIG['MYSQL_HOST'],
                           CONFIG['MYSQL_PORT'],
                           CONFIG['MYSQL_DB'])

In [4]:
def camel_to_snake_case(text):
    return re.sub(r'(?<!^)(?=[A-Z][a-z])', '_', text.replace(' ', '')).lower()


In [5]:
df = pd.read_csv('./data/fifa.csv', index_col=0, parse_dates=['Joined'])

In [6]:
df.head()

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [7]:
df.columns = [camel_to_snake_case(c) for c in df.columns]

In [8]:
df['gk'] = np.where(df.position == 'GK', df.overall, 0)

In [9]:
# Joined: Date
COLS = df.columns
COLS_POS = ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']
COLS_MONEY = ['value', 'wage', 'release_clause']
COLS_BOOL = ['real_face']
COLS_UNPROCESSED = set(COLS).difference(set([*COLS_POS, *COLS_MONEY, *COLS_BOOL]))

In [10]:
def process_position_overalls(text):
    if pd.isnull(text):
        return 0
    else:
        if isinstance(text, int):
            return text
        else:
            return int(text.split('+')[0])


def process_money_values(text):
    if pd.isnull(text):
        return 0
    else:
        if text.endswith('K'):
            return int(float(text[1:-1]) * 10**3)
        elif text.endswith('M'):
            return int(float(text[1:-1]) * 10**6)
        else:
            return int(float(text[1:]))


def process_bool_values(text):
    if pd.isnull(text):
        return 0
    else:
        if text == 'Yes':
            return 1
        elif text == 'No':
            return 0
        else:
            return None


def process_photo_links(text):
    start = 'https://cdn.sofifa.com/players'
    end = '19_60.png'
    id_str = str(text.split('/')[-1].split('.')[0]).zfill(6)
    return f'{start}/{id_str[:3]}/{id_str[3:]}/{end}'


def process_flag_links(text):
    pass

In [11]:
df = df[COLS_UNPROCESSED] \
        .merge(df[[*COLS_POS]].applymap(process_position_overalls), left_index=True, right_index=True) \
        .merge(df[[*COLS_MONEY]].applymap(process_money_values), left_index=True, right_index=True) \
        .merge(df[[*COLS_BOOL]].applymap(process_bool_values), left_index=True, right_index=True) \
        .assign(
            photo = lambda df: df['photo'].apply(process_photo_links),
            photo_id = lambda df: df['photo'].apply(lambda x: ''.join(x.split('/')[-3:-1])),
            position = lambda df: df['position'].str.lower()
        )


In [12]:
import requests

def scrape_player_photos(df, path):
    
    TOTAL = len(df)

    for index, row in df.iterrows():
        
        print(f'{index+1:>8} of {TOTAL:>8}', end='\r')
        response = requests.get(row['photo'])
        
        with open(f"{path}/{row['photo_id']}.png", 'wb') as f:
            f.write(response.content)

# PATH = '../../apps/fifa-team-builder/backend/assets/players'
# scrape_player_photos(df, PATH)

In [13]:
df['photo'] = df['photo_id'].apply(lambda x: f'/assets/players/{x}.png')

In [14]:
statement_insert_mysql = """
    CREATE TABLE players (
        id                         BIGINT PRIMARY KEY,
        name                       VARCHAR(25) NOT NULL,
        age                        INT NOT NULL,
        photo                      VARCHAR(50) NOT NULL,
        photo_id                   VARCHAR(10) NOT NULL,
        nationality                VARCHAR(25) NULL,
        flag                       VARCHAR(40) NOT NULL,
        overall                    INT NOT NULL,
        potential                  INT NOT NULL,
        club                       VARCHAR(40),
        club_logo                  VARCHAR(50) NOT NULL,
        value                      BIGINT NOT NULL,
        wage                       BIGINT NOT NULL,
        special                    INT NOT NULL,
        preferred_foot             VARCHAR(5),
        international_reputation   INT,
        weak_foot                  INT,
        skill_moves                INT,
        work_rate                  VARCHAR(15),
        body_type                  VARCHAR(20),
        real_face                  BOOLEAN,
        position                   VARCHAR(3),
        jersey_number              INT,
        joined                     DATE,
        loaned_from                VARCHAR(40),
        contract_valid_until       VARCHAR(15),
        height                     VARCHAR(5),
        weight                     VARCHAR(6),
        ls                         INT,
        st                         INT,
        rs                         INT,
        lw                         INT,
        lf                         INT,
        cf                         INT,
        rf                         INT,
        rw                         INT,
        lam                        INT,
        cam                        INT,
        ram                        INT,
        lm                         INT,
        lcm                        INT,
        cm                         INT,
        rcm                        INT,
        rm                         INT,
        lwb                        INT,
        ldm                        INT,
        cdm                        INT,
        rdm                        INT,
        rwb                        INT,
        lb                         INT,
        lcb                        INT,
        cb                         INT,
        rcb                        INT,
        rb                         INT,
        gk                         INT,
        crossing                   INT,
        finishing                  INT,
        heading_accuracy           INT,
        short_passing              INT,
        volleys                    INT,
        dribbling                  INT,
        curve                      INT,
        fk_accuracy                INT,
        long_passing               INT,
        ball_control               INT,
        acceleration               INT,
        sprint_speed               INT,
        agility                    INT,
        reactions                  INT,
        balance                    INT,
        shot_power                 INT,
        jumping                    INT,
        stamina                    INT,
        strength                   INT,
        long_shots                 INT,
        aggression                 INT,
        interceptions              INT,
        positioning                INT,
        vision                     INT,
        penalties                  INT,
        composure                  INT,
        marking                    INT,
        standing_tackle            INT,
        sliding_tackle             INT,
        gk_diving                  INT,
        gk_handling                INT,
        gk_kicking                 INT,
        gk_positioning             INT,
        gk_reflexes                INT,
        release_clause             BIGINT
    )
"""

model_players = {
    'id': tp.BIGINT,
    'name': tp.VARCHAR,
    'age': tp.INT,
    'photo': tp.VARCHAR,
    'photo_id': tp.VARCHAR,
    'nationality': tp.VARCHAR,
    'flag': tp.VARCHAR,
    'overall': tp.INT,
    'potential': tp.INT,
    'club': tp.VARCHAR,
    'club_logo': tp.VARCHAR,
    'value': tp.BIGINT,
    'wage': tp.BIGINT,
    'special': tp.INT,
    'preferred_foot': tp.VARCHAR,
    'international_reputation': tp.INT,
    'weak_foot': tp.INT,
    'skill_moves': tp.INT,
    'work_rate': tp.VARCHAR,
    'body_type': tp.VARCHAR,
    'real_face': tp.VARCHAR,
    'position': tp.VARCHAR,
    'jersey_number': tp.INT,
    'joined': tp.VARCHAR,
    'loaned_from': tp.VARCHAR,
    'contract_valid_until': tp.VARCHAR,
    'height': tp.VARCHAR,
    'weight': tp.VARCHAR,
    'ls': tp.INT,
    'st': tp.INT,
    'rs': tp.INT,
    'lw': tp.INT,
    'lf': tp.INT,
    'cf': tp.INT,
    'rf': tp.INT,
    'rw': tp.INT,
    'lam': tp.INT,
    'cam': tp.INT,
    'ram': tp.INT,
    'lm': tp.INT,
    'lcm': tp.INT,
    'cm': tp.INT,
    'rcm': tp.INT,
    'rm': tp.INT,
    'lwb': tp.INT,
    'ldm': tp.INT,
    'cdm': tp.INT,
    'rdm': tp.INT,
    'rwb': tp.INT,
    'lb': tp.INT,
    'lcb': tp.INT,
    'cb': tp.INT,
    'rcb': tp.INT,
    'rb': tp.INT,
    'gk': tp.INT,
    'crossing': tp.INT,
    'finishing': tp.INT,
    'heading_accuracy': tp.INT,
    'short_passing': tp.INT,
    'volleys': tp.INT,
    'dribbling': tp.INT,
    'curve': tp.INT,
    'fk_accuracy': tp.INT,
    'long_passing': tp.INT,
    'ball_control': tp.INT,
    'acceleration': tp.INT,
    'sprint_speed': tp.INT,
    'agility': tp.INT,
    'reactions': tp.INT,
    'balance': tp.INT,
    'shot_power': tp.INT,
    'jumping': tp.INT,
    'stamina': tp.INT,
    'strength': tp.INT,
    'long_shots': tp.INT,
    'aggression': tp.INT,
    'interceptions': tp.INT,
    'positioning': tp.INT,
    'vision': tp.INT,
    'penalties': tp.INT,
    'composure': tp.INT,
    'marking': tp.INT,
    'standing_tackle': tp.INT,
    'sliding_tackle': tp.INT,
    'gk_diving': tp.INT,
    'gk_handling': tp.INT,
    'gk_kicking': tp.INT,
    'gk_positioning': tp.INT,
    'gk_reflexes': tp.INT,
    'release_clause': tp.BIGINT,
}

In [16]:
with engine.connect() as con:
    con.execute("DROP TABLE IF EXISTS players")
    con.execute(statement_insert_mysql)

In [17]:
df.to_sql('players', con=engine, if_exists='append', index=False)

In [18]:
from models import Player

In [21]:
Session = sessionmaker(bind=engine)
session = Session()

In [22]:
NAME, CLUB, NAT = '', '', ''
rows_all = session.query(Player) \
                    .filter(Player.name.ilike(f'%{NAME}%')) \
                    .filter(Player.club.ilike(f'%{CLUB}%')) \
                    .filter(Player.nationality.ilike(f'%{NAT}%'))
rows_all.count()

17966